<a href="https://colab.research.google.com/github/ysuter/FHNW-BAI-ComputerVision/blob/main/W02/bildhistogramme_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Bildhistogramme - Von den Grundlagen bis zur Anwendung

**Lernziele:**
- ✅ Verstehen, was Bildhistogramme sind und warum sie wichtig sind
- ✅ RGB vs. Graustufen-Histogramme interpretieren können
- ✅ Histogram Equalization anwenden und verstehen
- ✅ CLAHE (Contrast Limited Adaptive Histogram Equalization) kennenlernen
- ✅ Histogram Matching und Contrast Stretching nutzen
- ✅ Praktische Anwendungen in der Bildverbesserung

---

## 🤔 Motivation: Warum Histogramme?

**Ein Histogramm zeigt auf einen Blick:**
- 📸 Ist das Bild über- oder unterbelichtet?
- 🎨 Nutzt das Bild den vollen Dynamikbereich?
- 🌓 Ist der Kontrast hoch oder niedrig?
- 🔍 Wo liegen die Hauptinformationen im Bild?

**Anwendungen:**
- Automatische Bildverbesserung (Smartphones!)
- Kontrastanpassung
- Details hervorheben
- Qualitätskontrolle (gleichmässige Belichtung prüfen)

---

## 📦 Setup & Installation

In [ ]:
# Bibliotheken installieren
!pip install opencv-python-headless scikit-image ipywidgets -q

# Imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Plotting-Einstellungen
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✅ Alle Bibliotheken geladen!")
print(f"OpenCV Version: {cv2.__version__}")
print(f"NumPy Version: {np.__version__}")

## 🎨 Hilfsfunktionen

In [ ]:
def load_image_from_url(url):
    """Lädt Bild von URL mit verbessertem Error Handling"""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Versuche Bild zu öffnen
        img = Image.open(BytesIO(response.content))

        # Konvertiere zu RGB falls nötig
        if img.mode not in ('RGB', 'L'):
            img = img.convert('RGB')

        img_array = np.array(img)

        if len(img_array.shape) == 2:  # Grayscale
            return cv2.cvtColor(img_array, cv2.COLOR_GRAY2BGR)
        elif len(img_array.shape) == 3:
            if img_array.shape[2] == 4:  # RGBA
                return cv2.cvtColor(img_array, cv2.COLOR_RGBA2BGR)
            elif img_array.shape[2] == 3:  # RGB
                return cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        return None
    except Exception as e:
        print(f"⚠️ Fehler: {str(e)[:60]}...")
        return None

def show_image_with_histogram(image, title="Bild", channels='bgr'):
    """
    Zeigt Bild und zugehöriges Histogramm
    channels: 'bgr' für Farbbild, 'gray' für Graustufenbild
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Bild anzeigen
    if len(image.shape) == 3:
        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else:
        axes[0].imshow(image, cmap='gray')
    axes[0].set_title(title, fontsize=12, fontweight='bold')
    axes[0].axis('off')

    # Histogramm berechnen und anzeigen
    if channels == 'gray':
        # Graustufenhistogramm
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        hist = cv2.calcHist([gray], [0], None, [256], [0, 256])
        axes[1].plot(hist, color='black', linewidth=2)
        axes[1].fill_between(range(256), hist.flatten(), alpha=0.3, color='gray')
        axes[1].set_xlim([0, 256])
        axes[1].set_title('Histogramm (Grayscale)', fontsize=12, fontweight='bold')
    else:
        # RGB/BGR Histogramm
        colors = ('b', 'g', 'r')
        color_names = ('Blau', 'Grün', 'Rot')
        for i, (color, name) in enumerate(zip(colors, color_names)):
            hist = cv2.calcHist([image], [i], None, [256], [0, 256])
            axes[1].plot(hist, color=color, linewidth=2, label=name, alpha=0.7)
        axes[1].set_xlim([0, 256])
        axes[1].set_title('Histogramm (RGB)', fontsize=12, fontweight='bold')
        axes[1].legend()

    axes[1].set_xlabel('Intensität (0-255)')
    axes[1].set_ylabel('Häufigkeit (Pixel-Anzahl)')
    axes[1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

print("✅ Hilfsfunktionen definiert!")

## 🖼️ Testbilder laden

In [ ]:
# Verschiedene Beispielbilder für unterschiedliche Histogramm-Charakteristiken
print("📥 Lade Beispielbilder...\n")

# Bild 1: Standard-Testbild (gut belichtet)
url_normal = "https://sipi.usc.edu/database/preview/misc/4.2.03.png"
img_normal = load_image_from_url(url_normal)

# Bild 2: Dunkles Bild erstellen (unterbelichtet)
if img_normal is not None:
    img_dark = (img_normal * 0.3).astype(np.uint8)

# Bild 3: Helles Bild erstellen (überbelichtet)
    img_bright = np.clip(img_normal * 1.5 + 50, 0, 255).astype(np.uint8)

# Bild 4: Kontrastarmes Bild
    img_low_contrast = ((img_normal - 128) * 0.3 + 128).astype(np.uint8)

    print("✅ Alle Testbilder erstellt!")
    print("\n📊 Wir haben jetzt 4 Bilder mit verschiedenen Eigenschaften:")
    print("   1. Normal belichtet")
    print("   2. Unterbelichtet (dunkel)")
    print("   3. Überbelichtet (hell)")
    print("   4. Niedriger Kontrast")
else:
    print("❌ Fehler beim Laden des Bildes")

---

# 📊 Teil 1: Histogramm-Grundlagen

## Was ist ein Histogramm?

**Definition:**
Ein Histogramm zeigt die Verteilung der Pixelintensitäten in einem Bild.

**Achsen:**
- **X-Achse**: Intensitätswerte (0-255 bei 8-bit Bildern)
  - 0 = Schwarz
  - 255 = Weiß
- **Y-Achse**: Anzahl der Pixel mit dieser Intensität

**Interpretation:**
- **Links (dunkle Werte)**: Schatten
- **Mitte**: Mitteltöne
- **Rechts (helle Werte)**: Lichter/Highlights

---

## 1.1 Vergleich verschiedener Belichtungen

In [ ]:
if img_normal is not None:
    print("📊 Vergleich der Histogramme bei unterschiedlichen Belichtungen\n")
    print("="*70)

    # Normal
    print("\n1️⃣ NORMAL BELICHTETES BILD:")
    print("   → Histogramm gut verteilt über gesamten Bereich")
    show_image_with_histogram(img_normal, "Normal belichtet", 'gray')

    # Dunkel
    print("\n2️⃣ UNTERBELICHTETES BILD (zu dunkel):")
    print("   → Histogramm nach LINKS verschoben (dunkle Werte)")
    print("   → Viele Pixel im Schattenbereich (0-100)")
    show_image_with_histogram(img_dark, "Unterbelichtet", 'gray')

    # Hell
    print("\n3️⃣ ÜBERBELICHTETES BILD (zu hell):")
    print("   → Histogramm nach RECHTS verschoben (helle Werte)")
    print("   → Viele Pixel im Highlight-Bereich (150-255)")
    show_image_with_histogram(img_bright, "Überbelichtet", 'gray')

    # Niedriger Kontrast
    print("\n4️⃣ NIEDRIGER KONTRAST:")
    print("   → Histogramm SCHMAL und ZENTRIERT")
    print("   → Nutzt nicht den vollen Dynamikbereich (0-255)")
    print("   → Bild wirkt 'flau' oder 'neblig'")
    show_image_with_histogram(img_low_contrast, "Niedriger Kontrast", 'gray')

### 💡 Interpretation lernen:

**Gutes Histogramm:**
- ✅ Gleichmäßig verteilt über 0-255
- ✅ Keine großen Lücken
- ✅ Nicht zu stark an den Rändern (Clipping vermeiden)

**Problematische Histogramme:**
- ❌ Zu weit links → unterbelichtet
- ❌ Zu weit rechts → überbelichtet
- ❌ Schmal und zentriert → niedriger Kontrast
- ❌ Spitzen an 0 oder 255 → Clipping (Informationsverlust!)

## 1.2 RGB-Farbhistogramme

In [ ]:
if img_normal is not None:
    print("🎨 RGB-Farbhistogramme\n")
    print("="*70)
    print("\nFarbbilder haben 3 separate Histogramme:")
    print("   🔴 Rot-Kanal")
    print("   🟢 Grün-Kanal")
    print("   🔵 Blau-Kanal\n")

    show_image_with_histogram(img_normal, "Farbbild mit RGB-Histogramm", 'bgr')

    print("\n💡 Farbstiche erkennen:")
    print("   - Wenn ein Kanal dominiert → Farbstich in dieser Farbe")
    print("   - Alle drei Kanäle ähnlich → neutrales Grau")
    print("   - Große Unterschiede zwischen Kanälen → bunte Szene")

## 1.3 Histogramm manuell berechnen

In [ ]:
# Zeigen wir, wie ein Histogramm intern berechnet wird
print("🔍 Histogramm-Berechnung im Detail\n")

# Kleines Beispielbild erstellen
small_img = np.array([
    [0, 50, 100],
    [50, 100, 150],
    [100, 150, 200]
], dtype=np.uint8)

print("Beispiel-Bild (3x3 Pixel):")
print(small_img)
print()

# Manuelle Berechnung
unique, counts = np.unique(small_img, return_counts=True)
print("Histogramm-Tabelle:")
print("-" * 30)
print(f"{'Intensität':<15} {'Anzahl Pixel':<15}")
print("-" * 30)
for intensity, count in zip(unique, counts):
    print(f"{intensity:<15} {count:<15}")
print("-" * 30)

# Visualisierung
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].imshow(small_img, cmap='gray', interpolation='nearest')
axes[0].set_title('Beispiel-Bild (3x3)', fontweight='bold')
axes[0].axis('off')

# Für alle 256 Werte
hist_full = np.zeros(256)
for intensity, count in zip(unique, counts):
    hist_full[intensity] = count

axes[1].bar(range(256), hist_full, width=1, color='gray', edgecolor='black')
axes[1].set_xlim([0, 255])
axes[1].set_title('Histogramm', fontweight='bold')
axes[1].set_xlabel('Intensität')
axes[1].set_ylabel('Anzahl Pixel')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 So funktioniert's:")
print("   1. Für jede Intensität 0-255 zählen wir die Pixel")
print("   2. Das Ergebnis ist das Histogramm")
print("   3. Summe aller Balken = Gesamtzahl Pixel im Bild")

---

# 🔧 Teil 2: Histogram Equalization (Standard)

## Was ist Histogram Equalization?

**Ziel:** Verbesserung des Kontrasts durch Umverteilung der Intensitätswerte

**Idee:**
- Spreize das Histogramm über den gesamten Bereich (0-255)
- Häufige Intensitäten werden weiter gespreizt
- Seltene Intensitäten werden komprimiert

**Mathematik (vereinfacht):**
1. Berechne **kumulative Verteilungsfunktion** (CDF)
2. Normalisiere CDF auf 0-255
3. Mappe alte Intensitäten auf neue

**Resultat:**
- Idealerweise: gleichmäßig verteiltes Histogramm
- Maximaler Kontrast

---

## 2.1 Histogram Equalization anwenden

In [ ]:
def apply_histogram_equalization(image):
    """
    Wendet Histogram Equalization auf Graustufenbild an
    """
    if len(image.shape) == 3:
        # Konvertiere zu Graustufen falls Farbbild
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    # Histogram Equalization
    equalized = cv2.equalizeHist(gray)

    return gray, equalized

def show_equalization_comparison(original, title=""):
    """
    Zeigt Vorher-Nachher-Vergleich von Histogram Equalization
    """
    gray, equalized = apply_histogram_equalization(original)

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Original Bild
    axes[0, 0].imshow(gray, cmap='gray')
    axes[0, 0].set_title('Original', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

    # Original Histogramm
    hist_orig = cv2.calcHist([gray], [0], None, [256], [0, 256])
    axes[0, 1].plot(hist_orig, color='black', linewidth=2)
    axes[0, 1].fill_between(range(256), hist_orig.flatten(), alpha=0.3, color='gray')
    axes[0, 1].set_xlim([0, 256])
    axes[0, 1].set_title('Original Histogramm', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Intensität')
    axes[0, 1].set_ylabel('Häufigkeit')
    axes[0, 1].grid(alpha=0.3)

    # Equalized Bild
    axes[1, 0].imshow(equalized, cmap='gray')
    axes[1, 0].set_title('Nach Histogram Equalization', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')

    # Equalized Histogramm
    hist_eq = cv2.calcHist([equalized], [0], None, [256], [0, 256])
    axes[1, 1].plot(hist_eq, color='blue', linewidth=2)
    axes[1, 1].fill_between(range(256), hist_eq.flatten(), alpha=0.3, color='blue')
    axes[1, 1].set_xlim([0, 256])
    axes[1, 1].set_title('Equalized Histogramm', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Intensität')
    axes[1, 1].set_ylabel('Häufigkeit')
    axes[1, 1].grid(alpha=0.3)

    if title:
        fig.suptitle(title, fontsize=14, fontweight='bold', y=1.00)

    plt.tight_layout()
    plt.show()

    # Statistiken
    print(f"📊 Statistiken:")
    print(f"   Original  → Min: {gray.min()}, Max: {gray.max()}, Mean: {gray.mean():.1f}, Std: {gray.std():.1f}")
    print(f"   Equalized → Min: {equalized.min()}, Max: {equalized.max()}, Mean: {equalized.mean():.1f}, Std: {equalized.std():.1f}")
    print(f"\n   ➡️ Dynamikbereich wurde von [{gray.min()}-{gray.max()}] auf [{equalized.min()}-{equalized.max()}] erweitert!")

In [ ]:
if img_dark is not None:
    print("🌓 BEISPIEL 1: Unterbelichtetes Bild verbessern\n")
    print("="*70)
    show_equalization_comparison(img_dark, "Histogram Equalization - Unterbelichtetes Bild")
    print("\n💡 Beobachtung:")
    print("   ✅ Dunkles Bild wurde aufgehellt")
    print("   ✅ Details in Schatten sind jetzt sichtbar")
    print("   ✅ Histogramm ist breiter verteilt")
    print("   ⚠️  Aber: Kann auch Rauschen verstärken!")

In [ ]:
if img_low_contrast is not None:
    print("\n\n📉 BEISPIEL 2: Kontrastarmes Bild verbessern\n")
    print("="*70)
    show_equalization_comparison(img_low_contrast, "Histogram Equalization - Niedriger Kontrast")
    print("\n💡 Beobachtung:")
    print("   ✅ Kontrast deutlich erhöht")
    print("   ✅ Bild wirkt nicht mehr 'flau'")
    print("   ✅ Histogramm nutzt vollen Bereich 0-255")

## 2.2 Histogram Equalization für Farbbilder

In [ ]:
def equalize_color_image(image):
    """
    Histogram Equalization für Farbbilder
    Arbeitet im YCrCb-Farbraum (nur Luminanz wird equalized)
    """
    # Konvertiere BGR zu YCrCb
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Equalize nur den Y-Kanal (Luminanz/Helligkeit)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])

    # Zurück zu BGR
    result = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

    return result

if img_dark is not None:
    print("🎨 Histogram Equalization für FARBBILDER\n")
    print("="*70)
    print("\nMethode: Equalization im YCrCb-Farbraum")
    print("   → Nur Luminanz (Y-Kanal) wird equalized")
    print("   → Farben (Cr, Cb) bleiben erhalten\n")

    # Dunkles Farbbild equalisieren
    equalized_color = equalize_color_image(img_dark)

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    axes[0].imshow(cv2.cvtColor(img_dark, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original (dunkel)', fontsize=12, fontweight='bold')
    axes[0].axis('off')

    axes[1].imshow(cv2.cvtColor(equalized_color, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Nach Histogram Equalization', fontsize=12, fontweight='bold')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    print("\n💡 Warum YCrCb statt RGB?")
    print("   - Trennung von Helligkeit (Y) und Farbe (Cr, Cb)")
    print("   - Wenn wir RGB-Kanäle einzeln equalisieren → Farbverschiebungen!")
    print("   - YCrCb: Nur Helligkeit ändern, Farben bleiben natürlich")

## 2.3 Probleme von Standard Histogram Equalization

In [ ]:
# Demonstriere Überverstärkung bei normalem Bild
if img_normal is not None:
    print("⚠️ PROBLEM: Überverstärkung bei bereits gutem Bild\n")
    print("="*70)

    show_equalization_comparison(img_normal, "Problem: Equalization auf gut belichtetem Bild")

    print("\n❌ Probleme der Standard-Equalization:")
    print("   1. Rauschen wird verstärkt")
    print("   2. Lokale Details können verloren gehen")
    print("   3. Übertriebener Kontrast (unnatürlich)")
    print("   4. Artefakte in gleichmäßigen Bereichen")
    print("\n✅ Ansatz: CLAHE (Contrast Limited Adaptive Histogram Equalization)")

---

# 🎯 Teil 3: CLAHE (Contrast Limited Adaptive Histogram Equalization)

## Was ist CLAHE?

**Verbesserungen gegenüber Standard-Equalization:**

1. **Adaptive** (lokal statt global)
   - Bild wird in kleine Kacheln unterteilt (z.B. 8×8)
   - Equalization wird für jede Kachel separat durchgeführt
   - Grenzen werden interpoliert (sanfte Übergänge)

2. **Contrast Limited** (begrenzte Verstärkung)
   - Clip Limit: Maximale Verstärkung wird begrenzt
   - Verhindert Rauschverstärkung
   - Natürlichere Ergebnisse

**Parameter:**
- **clipLimit**: Schwellwert für Kontrastverstärkung (typisch: 2.0-4.0)
- **tileGridSize**: Größe der Kacheln (typisch: 8×8)

---

## 3.1 CLAHE anwenden

In [ ]:
def apply_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    """
    Wendet CLAHE auf Graustufenbild an
    """
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    # CLAHE-Objekt erstellen
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    # Anwenden
    clahe_img = clahe.apply(gray)

    return gray, clahe_img

def compare_equalization_methods(image, title=""):
    """
    Vergleicht Standard-Equalization mit CLAHE
    """
    # Graustufen
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    # Standard Equalization
    eq_standard = cv2.equalizeHist(gray)

    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    eq_clahe = clahe.apply(gray)

    # Visualisierung
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))

    # Zeile 1: Bilder
    axes[0, 0].imshow(gray, cmap='gray')
    axes[0, 0].set_title('Original', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

    axes[0, 1].imshow(eq_standard, cmap='gray')
    axes[0, 1].set_title('Standard Equalization', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')

    axes[0, 2].imshow(eq_clahe, cmap='gray')
    axes[0, 2].set_title('CLAHE', fontsize=12, fontweight='bold')
    axes[0, 2].axis('off')

    # Zeile 2: Histogramme
    for idx, (img, label) in enumerate([(gray, 'Original'),
                                         (eq_standard, 'Standard'),
                                         (eq_clahe, 'CLAHE')]):
        hist = cv2.calcHist([img], [0], None, [256], [0, 256])
        color = 'black' if idx == 0 else 'blue' if idx == 1 else 'green'
        axes[1, idx].plot(hist, color=color, linewidth=2)
        axes[1, idx].fill_between(range(256), hist.flatten(), alpha=0.3, color=color)
        axes[1, idx].set_xlim([0, 256])
        axes[1, idx].set_title(f'Histogramm: {label}', fontsize=11, fontweight='bold')
        axes[1, idx].set_xlabel('Intensität')
        axes[1, idx].set_ylabel('Häufigkeit')
        axes[1, idx].grid(alpha=0.3)

    if title:
        fig.suptitle(title, fontsize=14, fontweight='bold', y=0.98)

    plt.tight_layout()
    plt.show()

if img_dark is not None:
    print("🔍 Vergleich: Standard Equalization vs. CLAHE\n")
    print("="*70)
    compare_equalization_methods(img_dark, "Vergleich der Methoden")

    print("\n📊 Beobachtungen:")
    print("\n   Standard Equalization:")
    print("   ✅ Maximale Kontrasterhöhung")
    print("   ❌ Kann übertrieben wirken")
    print("   ❌ Verstärkt Rauschen stark")

    print("\n   CLAHE:")
    print("   ✅ Natürlichere Ergebnisse")
    print("   ✅ Weniger Rauschverstärkung")
    print("   ✅ Bessere lokale Details")
    print("   ⚠️  Etwas weniger Kontrast als Standard")

## 3.2 Interaktive CLAHE-Parameter

In [ ]:
# Interaktive Widgets für CLAHE-Parameter
if img_dark is not None:
    print("🎛️ Experimentieren Sie mit CLAHE-Parametern!\n")

    clip_slider = widgets.FloatSlider(
        value=2.0,
        min=1.0,
        max=10.0,
        step=0.5,
        description='Clip Limit:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )

    tile_slider = widgets.IntSlider(
        value=8,
        min=2,
        max=32,
        step=2,
        description='Tile Size:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )

    output_clahe = widgets.Output()

    def update_clahe(change):
        with output_clahe:
            clear_output(wait=True)

            gray = cv2.cvtColor(img_dark, cv2.COLOR_BGR2GRAY)

            # CLAHE anwenden
            tile_size = (tile_slider.value, tile_slider.value)
            clahe = cv2.createCLAHE(clipLimit=clip_slider.value,
                                   tileGridSize=tile_size)
            result = clahe.apply(gray)

            # Anzeigen
            fig, axes = plt.subplots(1, 2, figsize=(14, 6))

            axes[0].imshow(gray, cmap='gray')
            axes[0].set_title('Original', fontsize=12)
            axes[0].axis('off')

            axes[1].imshow(result, cmap='gray')
            axes[1].set_title(
                f'CLAHE (Clip={clip_slider.value:.1f}, Tile={tile_size[0]}×{tile_size[1]})',
                fontsize=12
            )
            axes[1].axis('off')

            plt.tight_layout()
            plt.show()

    clip_slider.observe(update_clahe, names='value')
    tile_slider.observe(update_clahe, names='value')

    display(clip_slider, tile_slider, output_clahe)
    update_clahe(None)

    print("\n💡 Parameter-Tipps:")
    print("   📊 Clip Limit:")
    print("      • Niedrig (1-2): Sanfte Verstärkung, wenig Rauschen")
    print("      • Mittel (2-4): Guter Kompromiss (Standard)")
    print("      • Hoch (>4): Starke Verstärkung, mehr Rauschen")
    print("\n   🔲 Tile Size:")
    print("      • Klein (2-4): Sehr lokal, gut für Details")
    print("      • Mittel (8-16): Standard, gute Balance")
    print("      • Groß (>16): Mehr wie globale Equalization")

---

# 📏 Teil 4: Contrast Stretching (Lineare Normalisierung)

## Was ist Contrast Stretching?

**Einfachste Methode zur Kontrastverbesserung:**

**Idee:**
- Finde minimale und maximale Intensität im Bild
- Strecke diesen Bereich auf 0-255

**Formel:**
```
new_pixel = (pixel - min) × (255 / (max - min))
```

**Unterschied zu Histogram Equalization:**
- ✅ Einfacher (nur lineare Skalierung)
- ✅ Keine Histogramm-Änderung, nur Streckung
- ❌ Weniger effektiv bei ungleichmäßiger Verteilung

---

In [ ]:
def contrast_stretching(image):
    """
    Contrast Stretching (Min-Max Normalisierung)
    """
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    # Finde Min und Max
    min_val = gray.min()
    max_val = gray.max()

    # Strecke auf 0-255
    stretched = ((gray - min_val) / (max_val - min_val) * 255).astype(np.uint8)

    return gray, stretched, min_val, max_val

if img_low_contrast is not None:
    print("📏 Contrast Stretching\n")
    print("="*70)

    gray, stretched, min_val, max_val = contrast_stretching(img_low_contrast)

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # Original
    axes[0, 0].imshow(gray, cmap='gray')
    axes[0, 0].set_title('Original (niedriger Kontrast)', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

    # Original Histogramm
    hist_orig = cv2.calcHist([gray], [0], None, [256], [0, 256])
    axes[0, 1].plot(hist_orig, color='black', linewidth=2)
    axes[0, 1].axvline(min_val, color='red', linestyle='--', label=f'Min={min_val}')
    axes[0, 1].axvline(max_val, color='blue', linestyle='--', label=f'Max={max_val}')
    axes[0, 1].fill_between(range(256), hist_orig.flatten(), alpha=0.3, color='gray')
    axes[0, 1].set_xlim([0, 256])
    axes[0, 1].set_title('Original Histogramm', fontsize=12, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)

    # Stretched
    axes[1, 0].imshow(stretched, cmap='gray')
    axes[1, 0].set_title('Nach Contrast Stretching', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')

    # Stretched Histogramm
    hist_stretched = cv2.calcHist([stretched], [0], None, [256], [0, 256])
    axes[1, 1].plot(hist_stretched, color='green', linewidth=2)
    axes[1, 1].fill_between(range(256), hist_stretched.flatten(), alpha=0.3, color='green')
    axes[1, 1].set_xlim([0, 256])
    axes[1, 1].set_title('Stretched Histogramm', fontsize=12, fontweight='bold')
    axes[1, 1].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f"\n📊 Transformation:")
    print(f"   Alter Bereich: [{min_val} - {max_val}] ({max_val - min_val} Werte genutzt)")
    print(f"   Neuer Bereich: [0 - 255] (voller Dynamikbereich)")
    print(f"   Streckfaktor: {255 / (max_val - min_val):.2f}")

## 4.1 Vergleich aller Methoden

In [ ]:
def compare_all_methods(image):
    """
    Vergleicht alle Kontrast-Verbesserungsmethoden
    """
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()

    # Alle Methoden anwenden
    eq_standard = cv2.equalizeHist(gray)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    eq_clahe = clahe.apply(gray)

    min_val, max_val = gray.min(), gray.max()
    stretched = ((gray - min_val) / (max_val - min_val) * 255).astype(np.uint8)

    # Visualisierung
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))

    methods = [
        (gray, 'Original', 'black'),
        (stretched, 'Contrast Stretching', 'green'),
        (eq_standard, 'Standard Equalization', 'blue'),
        (eq_clahe, 'CLAHE', 'red')
    ]

    for idx, (img, title, color) in enumerate(methods):
        # Bild
        axes[0, idx].imshow(img, cmap='gray')
        axes[0, idx].set_title(title, fontsize=11, fontweight='bold')
        axes[0, idx].axis('off')

        # Histogramm
        hist = cv2.calcHist([img], [0], None, [256], [0, 256])
        axes[1, idx].plot(hist, color=color, linewidth=2)
        axes[1, idx].fill_between(range(256), hist.flatten(), alpha=0.3, color=color)
        axes[1, idx].set_xlim([0, 256])
        axes[1, idx].set_xlabel('Intensität', fontsize=9)
        axes[1, idx].set_ylabel('Häufigkeit', fontsize=9)
        axes[1, idx].grid(alpha=0.3)
        axes[1, idx].tick_params(labelsize=8)

    plt.suptitle('Vergleich aller Methoden', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

if img_low_contrast is not None:
    print("🔍 Umfassender Vergleich aller Methoden\n")
    print("="*70)
    compare_all_methods(img_low_contrast)

    print("\n📊 Zusammenfassung:\n")
    print("   1️⃣ Contrast Stretching:")
    print("      ✅ Einfachste Methode")
    print("      ✅ Schnell")
    print("      ❌ Nur bei schmalen Histogrammen effektiv")

    print("\n   2️⃣ Standard Histogram Equalization:")
    print("      ✅ Maximaler Kontrast")
    print("      ❌ Kann übertrieben wirken")
    print("      ❌ Verstärkt Rauschen")

    print("\n   3️⃣ CLAHE:")
    print("      ✅ Beste Gesamtqualität (meist!)")
    print("      ✅ Lokale Anpassung")
    print("      ✅ Kontrollierbare Verstärkung")
    print("      ⚠️  Rechenintensiver")

---

# 🎨 Teil 5: Histogram Matching (Spezifikation)

## Was ist Histogram Matching?

**Ziel:** Transformiere Histogramm eines Bildes, sodass es einem Referenz-Histogramm ähnelt

**Anwendungen:**
- Farbkorrektur (angleichen an Referenzbild)
- Stil-Transfer (Histogramm eines Kunstwerks übernehmen)
- Normalisierung in Bildserien

**Algorithmus:**
1. Berechne CDF von Quell- und Zielbild
2. Finde Mapping zwischen beiden CDFs
3. Wende Mapping auf Quellbild an

---

In [ ]:
def histogram_matching(source, reference):
    """
    Histogram Matching: Transformiert Histogramm von 'source' zu 'reference'
    """
    # Zu Graustufen falls nötig
    if len(source.shape) == 3:
        source = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
    if len(reference.shape) == 3:
        reference = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY)

    # Berechne Histogramme
    hist_source = cv2.calcHist([source], [0], None, [256], [0, 256]).flatten()
    hist_ref = cv2.calcHist([reference], [0], None, [256], [0, 256]).flatten()

    # Berechne kumulative Verteilungsfunktionen (CDF)
    cdf_source = hist_source.cumsum()
    cdf_source = cdf_source / cdf_source[-1]  # Normalisieren auf [0, 1]

    cdf_ref = hist_ref.cumsum()
    cdf_ref = cdf_ref / cdf_ref[-1]

    # Erstelle Lookup-Table für Mapping
    lookup_table = np.zeros(256, dtype=np.uint8)

    for i in range(256):
        # Finde nächsten Wert in Referenz-CDF
        diff = np.abs(cdf_ref - cdf_source[i])
        lookup_table[i] = np.argmin(diff)

    # Wende Mapping an
    matched = cv2.LUT(source, lookup_table)

    return matched

if img_dark is not None and img_bright is not None:
    print("🎨 Histogram Matching Demo\n")
    print("="*70)
    print("\nWir passen das Histogramm des dunklen Bildes an das helle Bild an.\n")

    # Matching durchführen
    matched = histogram_matching(img_dark, img_bright)

    # Zu Graustufen
    dark_gray = cv2.cvtColor(img_dark, cv2.COLOR_BGR2GRAY)
    bright_gray = cv2.cvtColor(img_bright, cv2.COLOR_BGR2GRAY)

    # Visualisierung
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))

    # Zeile 1: Bilder
    axes[0, 0].imshow(dark_gray, cmap='gray')
    axes[0, 0].set_title('Quellbild (dunkel)', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')

    axes[0, 1].imshow(matched, cmap='gray')
    axes[0, 1].set_title('Nach Matching', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')

    axes[0, 2].imshow(bright_gray, cmap='gray')
    axes[0, 2].set_title('Referenzbild (hell)', fontsize=12, fontweight='bold')
    axes[0, 2].axis('off')

    # Zeile 2: Histogramme
    for idx, (img, label, color) in enumerate([
        (dark_gray, 'Quelle', 'black'),
        (matched, 'Matched', 'purple'),
        (bright_gray, 'Referenz', 'orange')
    ]):
        hist = cv2.calcHist([img], [0], None, [256], [0, 256])
        axes[1, idx].plot(hist, color=color, linewidth=2)
        axes[1, idx].fill_between(range(256), hist.flatten(), alpha=0.3, color=color)
        axes[1, idx].set_xlim([0, 256])
        axes[1, idx].set_title(f'Histogramm: {label}', fontsize=11, fontweight='bold')
        axes[1, idx].set_xlabel('Intensität')
        axes[1, idx].set_ylabel('Häufigkeit')
        axes[1, idx].grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("\n💡 Beobachtung:")
    print("   → Das 'Matched' Histogramm ähnelt dem 'Referenz' Histogramm")
    print("   → Das Bild wurde entsprechend transformiert")
    print("   → Nützlich für: Farbkorrektur, Stil-Transfer, Bildnormalisierung")

---

# 📸 Teil 6: Praktische Anwendungen

## Eigenes Bild hochladen und bearbeiten

In [ ]:
from google.colab import files

print("📤 Laden Sie Ihr eigenes Bild hoch!\n")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    user_img = cv2.imread(filename)

    if user_img is not None:
        print(f"\n✅ Bild '{filename}' geladen!")
        print(f"   Größe: {user_img.shape[1]} × {user_img.shape[0]} Pixel\n")

        # Alle Methoden anwenden
        print("🔧 Wende alle Verbesserungsmethoden an...\n")
        compare_all_methods(user_img)

        print("\n💾 Möchten Sie ein verbessertes Bild speichern?")
        print("   Führen Sie die nächste Zelle aus!")
    else:
        print("❌ Fehler beim Laden des Bildes")
else:
    print("⚠️ Kein Bild hochgeladen")

## Verbessertes Bild speichern

In [ ]:
# Wählen Sie die gewünschte Methode
if 'user_img' in globals() and user_img is not None:
    gray_user = cv2.cvtColor(user_img, cv2.COLOR_BGR2GRAY)

    # CLAHE anwenden (meist beste Wahl)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    improved = clahe.apply(gray_user)

    # Speichern
    output_filename = 'improved_image.png'
    cv2.imwrite(output_filename, improved)

    print(f"✅ Verbessertes Bild gespeichert als '{output_filename}'")

    # Download
    files.download(output_filename)
    print("📥 Download gestartet!")

    # Anzeigen
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    axes[0].imshow(gray_user, cmap='gray')
    axes[0].set_title('Vorher', fontsize=14, fontweight='bold')
    axes[0].axis('off')

    axes[1].imshow(improved, cmap='gray')
    axes[1].set_title('Nachher (CLAHE)', fontsize=14, fontweight='bold')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Bitte laden Sie zuerst ein Bild in der vorherigen Zelle hoch!")

---

# 📚 Teil 7: Zusammenfassung & Entscheidungshilfe

## Welche Methode wann verwenden?

In [ ]:
decision_tree = """
<style>
table {
    width: 100%;
    border-collapse: collapse;
    margin: 20px 0;
}
th, td {
    border: 1px solid #ddd;
    padding: 12px;
    text-align: left;
}
th {
    background-color: #2196F3;
    color: white;
}
tr:nth-child(even) {
    background-color: #f2f2f2;
}
.use-case { font-weight: bold; color: #1976D2; }
.method { font-weight: bold; color: #388E3C; }
</style>

<h2>🎯 Entscheidungsbaum: Welche Methode?</h2>

<table>
  <tr>
    <th>Problem</th>
    <th>Empfohlene Methode</th>
    <th>Warum?</th>
  </tr>
  <tr>
    <td class="use-case">📸 Unterbelichtetes Foto</td>
    <td class="method">CLAHE oder Standard Equalization</td>
    <td>Bringt Details in Schatten zum Vorschein</td>
  </tr>
  <tr>
    <td class="use-case">☀️ Überbelichtetes Foto</td>
    <td class="method">CLAHE</td>
    <td>Reduziert Überbelichtung, erhält Details</td>
  </tr>
  <tr>
    <td class="use-case">🌫️ Neblig/Kontrastarmes Bild</td>
    <td class="method">Standard Equalization oder CLAHE</td>
    <td>Maximiert Kontrast, entfernt "Nebel"</td>
  </tr>
  <tr>
    <td class="use-case">🏥 Medizinisches Bild (Röntgen, MRT)</td>
    <td class="method">CLAHE</td>
    <td>Beste lokale Kontrastverbesserung ohne Überverstärkung</td>
  </tr>
  <tr>
    <td class="use-case">🛰️ Satellitenbild</td>
    <td class="method">CLAHE oder Histogram Matching</td>
    <td>Lokale Details wichtig, Matching für Zeitreihen</td>
  </tr>
  <tr>
    <td class="use-case">📱 Smartphone-Automatik</td>
    <td class="method">CLAHE (Echtzeit-optimiert)</td>
    <td>Gute Balance, natürliches Ergebnis</td>
  </tr>
  <tr>
    <td class="use-case">🎨 Künstlerischer Effekt</td>
    <td class="method">Standard Equalization</td>
    <td>Dramatischer Kontrast, expressiv</td>
  </tr>
  <tr>
    <td class="use-case">📊 Dokument-Scan</td>
    <td class="method">Contrast Stretching oder Standard Equalization</td>
    <td>Text muss lesbar sein, simpel reicht oft</td>
  </tr>
  <tr>
    <td class="use-case">🎬 Video-Normalisierung</td>
    <td class="method">Histogram Matching</td>
    <td>Konsistenz über Frames hinweg</td>
  </tr>
  <tr>
    <td class="use-case">🔬 Mikroskopie</td>
    <td class="method">CLAHE</td>
    <td>Feine Details sichtbar machen ohne Artefakte</td>
  </tr>
</table>

<h3>⚡ Quick Decision:</h3>
<ul>
  <li><strong>Schnell & einfach?</strong> → Contrast Stretching</li>
  <li><strong>Maximum Kontrast?</strong> → Standard Histogram Equalization</li>
  <li><strong>Beste Qualität?</strong> → CLAHE (meist!)</li>
  <li><strong>Farbanpassung?</strong> → Histogram Matching</li>
</ul>

<h3>🎓 Faustregel:</h3>
<p><strong>Wenn unsicher → Probiere CLAHE mit clipLimit=2.0, tileGridSize=(8,8)</strong></p>
<p>Das ist in 80% der Fälle die beste Wahl!</p>
"""

display(HTML(decision_tree))

## Wichtigste Konzepte - Cheat Sheet

In [ ]:
cheat_sheet = """
<style>
.code-box {
    background-color: #f5f5f5;
    border-left: 4px solid #2196F3;
    padding: 10px;
    margin: 10px 0;
    font-family: monospace;
}
.tip {
    background-color: #fff3cd;
    border-left: 4px solid #ffc107;
    padding: 10px;
    margin: 10px 0;
}
</style>

<h2>📝 Code Cheat Sheet</h2>

<h3>1. Histogramm berechnen und anzeigen:</h3>
<div class="code-box">
# Histogramm berechnen<br>
hist = cv2.calcHist([image], [0], None, [256], [0, 256])<br>
<br>
# Anzeigen<br>
plt.plot(hist)<br>
plt.xlim([0, 256])<br>
plt.show()
</div>

<h3>2. Standard Histogram Equalization:</h3>
<div class="code-box">
# Graustufenbild<br>
equalized = cv2.equalizeHist(gray_image)<br>
<br>
# Farbbild (YCrCb-Methode)<br>
ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)<br>
ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])<br>
result = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)
</div>

<h3>3. CLAHE:</h3>
<div class="code-box">
# CLAHE-Objekt erstellen<br>
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))<br>
<br>
# Anwenden<br>
result = clahe.apply(gray_image)
</div>

<div class="tip">
💡 <strong>Tipp:</strong> clipLimit kontrolliert die Stärke:<br>
&nbsp;&nbsp;&nbsp;• 1.0-2.0: Sanft (natürlich)<br>
&nbsp;&nbsp;&nbsp;• 2.0-4.0: Standard (empfohlen)<br>
&nbsp;&nbsp;&nbsp;• >4.0: Stark (kann übertrieben wirken)
</div>

<h3>4. Contrast Stretching:</h3>
<div class="code-box">
min_val = image.min()<br>
max_val = image.max()<br>
stretched = ((image - min_val) / (max_val - min_val) * 255).astype(np.uint8)
</div>

<h3>5. RGB-Histogramme (alle Kanäle):</h3>
<div class="code-box">
colors = ('b', 'g', 'r')<br>
for i, color in enumerate(colors):<br>
&nbsp;&nbsp;&nbsp;&nbsp;hist = cv2.calcHist([image], [i], None, [256], [0, 256])<br>
&nbsp;&nbsp;&nbsp;&nbsp;plt.plot(hist, color=color)
</div>
"""

display(HTML(cheat_sheet))

---

## 🎯 Lernziel-Check

Kannst du die folgenden Fragen beantworten?

### Grundlagen:
☑️ Was zeigt ein Histogramm und wie interpretiert man es?

☑️ Woran erkennt man ein unter-/überbelichtetes Bild im Histogramm?

☑️ Was bedeutet "voller Dynamikbereich"?

### Methoden:
☑️ Wie funktioniert Histogram Equalization im Prinzip?

☑️ Was ist der Unterschied zwischen Standard-Equalization und CLAHE?

☑️ Wann verwendet man Contrast Stretching vs. Equalization?

☑️ Warum verwendet man bei Farbbildern den YCrCb-Farbraum?

### Anwendung:
☑️ Welche Methode für unterbelichtete Fotos?

☑️ Welche Parameter beeinflussen CLAHE und wie?

☑️ Wann würde man Histogram Matching verwenden?

---

## 📚 Weiterführende Themen

- **Lokale Histogramm-Equalization** (sliding window)
- **Multi-Histogramm-Equalization** (für Farbbilder)
- **Histogramm-basierte Segmentierung** (Otsu's Method)
- **Tone Mapping** (HDR → LDR)
- **Gamma-Korrektur** (nicht-lineare Transformation)
